## Limpieza de datos y tratamiento de los valores nulos

El objetivo es limpiar y estandarizar el dataset de empleados de MetaCortex (tratamiento de nulos y duplicados en `nif`, inferencia/completado de `genero` y `sexo_biologico`, normalización de `activo` y `horas_semanales`, imputación de `pago` y `bonus`, selección/renombrado de columnas clave) y exportar el dataset final.


In [34]:
# Cargamos el dataset previamente tratado
import pandas as pd
df = pd.read_feather('../data/02_intermediate/empleados_metacortex.ftr')
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0
2,1979-09-19,2004-03-01,Fran Garcia,M,29015,None,None,Finanzas,True,40,NaN,NaN
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,None,Finanzas,False,40h,3000.0,1074.0
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,None,IT,True,45,6000.0,NaN
6,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,38,5000.0,938.0
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,NaN


In [35]:
# Resumen de tipos de datos y valores nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fecha_nacimiento  9 non-null      datetime64[ns]
 1   fecha_alta        9 non-null      datetime64[ns]
 2   nombre            9 non-null      object        
 3   sexo_biologico    7 non-null      object        
 4   codigo_postal     9 non-null      int64         
 5   nif               8 non-null      object        
 6   genero            6 non-null      object        
 7   departamento      9 non-null      object        
 8   activo            9 non-null      bool          
 9   horas_semanales   9 non-null      object        
 10  bonus             5 non-null      float64       
 11  pago              6 non-null      float64       
dtypes: bool(1), datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 933.0+ bytes


## Tratamiento de valores nulos
Utilizaremos las siguientes funciones para operar con los valores nulos:
- isnull()
- dropna()
- duplicated()
- drop_duplicates()

In [ ]:
df.isnull().sum()       # Conteo de valores nulos por columna

fecha_nacimiento    0
fecha_alta          0
nombre              0
sexo_biologico      2
codigo_postal       0
nif                 1
genero              3
departamento        0
activo              0
horas_semanales     0
bonus               4
pago                3
dtype: int64

### Tratamiento de valores nulos: Borrado
Por ejemplo, el valor nif es un dato que no puede inferirse, y a efectos de reporte es imprescindible al ser la clave única. En este caso, la opción va a ser eliminar el registro completo

In [ ]:
df.dropna(subset=['nif'], inplace=True) # Elimina filas con nif nulo (clave única)
df.isnull().sum()                       # Recuento de nulos tras eliminar nif nulos

fecha_nacimiento    0
fecha_alta          0
nombre              0
sexo_biologico      2
codigo_postal       0
nif                 0
genero              2
departamento        0
activo              0
horas_semanales     0
bonus               3
pago                2
dtype: int64

Si el valor es único (como nif) busca duplicados:

In [ ]:
df_test = df[df['nif'].duplicated(keep=False)]
df_test

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago
6,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,38,5000.0,938.0
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0


Valores posibles del parámetro **keep**:
- `keep='first'` (por defecto)
- `keep='last'`
- `keep=False`

La función `drop_duplicates()` funciona en el mismo sentido eliminandolos en vez de seleccionandolos.

En nuestros datos, vamos a dejar la última ocurrencia por nif al entender que es la más reciente:

In [ ]:
df.drop_duplicates(subset=['nif'], keep='last', inplace=True)
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,None,Finanzas,False,40h,3000.0,1074.0
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,None,IT,True,45,6000.0,NaN
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,NaN


## Rellenando los valores nulos o vacíos
La función **fillna** dispone de los siguientes métodos para rellenar:

- `df.fillna(method='ffill')`      # Forward fill (propaga hacia adelante)
- `df.fillna(method='bfill')`      # Backward fill (propaga hacia atrás)
- `df.fillna(method='pad')`        # Igual que ffill
- `df.fillna(method='backfill')`

Tambien podemos definir límites:
- `df.fillna(method='ffill', limit=2)`


In [ ]:
df['pago'] = df['pago'].fillna(0)       # 0 si está vacio
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,None,Finanzas,False,40h,3000.0,1074.0
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,None,IT,True,45,6000.0,0.0
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0


Creamos el campo 'bonus1' con la media global de 'bonus'

In [ ]:
df['bonus1'] = df['bonus'].fillna(df['bonus'].mean())
df

Creamos el campo 'bonus2' con la media por departamento

In [ ]:
df['bonus2'] = df.groupby('departamento')['bonus'].transform(
    lambda x: x.fillna(x.mean())
)
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,None,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,None,IT,True,45,6000.0,0.0,6000.0,6000.0
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0


In [ ]:
df.isnull().sum()               # Verifica nulos pendientes

fecha_nacimiento    0
fecha_alta          0
nombre              0
sexo_biologico      2
codigo_postal       0
nif                 0
genero              2
departamento        0
activo              0
horas_semanales     0
bonus               3
pago                0
bonus1              0
bonus2              0
dtype: int64

## Librerías de python que averiguan el genero del nombre de pila
- La librería **gender_guesser** 
- El dataset de nombres **names_dataset** alimentado por 730K nombres obtenidos de 533M de usuarios de facebook. (105 paises)


In [44]:
# gender_guesser: librería de python que averigua el género de un nombre
# Detección de género por nombre (heurística)
import gender_guesser.detector as gender

d = gender.Detector()
print(d.get_gender('Andrea'))  # Devuelve 'female'

# Para ser más específico con el país
print(d.get_gender('Andrea', 'italy'))  # Si soporta países específicos (mas de 50)

female
male


In [45]:
# Dataset de nombres para estimar género
from names_dataset import NameDataset, NameWrapper
nd = NameDataset()

In [46]:
# Ejemplo de inferencia de género con NameDataset
print(NameWrapper(nd.search('Andrea')).gender)

Male


La opción mas recomendada es cargar un dataset para cada genero y luego chequear el nombre si existe.
<pre>
from names_dataset import NameDataset, NameWrapper
nd = NameDataset()
listado_male = nd.get_top_names(n=1000, country_alpha2='ES', gender='male')
listado_female = nd.get_top_names(n=1000, country_alpha2='ES', gender='female')
print(listado_male)
</pre>

### Alimentando el genero en los valores nulos de nuestros datos

In [47]:
# Valores distintos en la columna genero
df.genero.unique()

array(['F', 'M', None], dtype=object)

In [48]:
# Para un chequeo visual, extraemos el nombre de pila y hacemos un contador
df['nombre_pila'] = df['nombre'].str.split(' ').str[0]
df.groupby(['nombre_pila','genero']).count()

,,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2
nombre_pila,genero,,,,,,,,,,,,,
Ana,F,1,1,1,1,1,1,1,1,1,1,1,1,1
Carlos,F,1,1,1,1,1,1,1,1,1,0,1,1,1
Elena,F,1,1,1,1,1,1,1,1,1,1,1,1,1
Gorka,M,1,1,1,1,1,1,1,1,1,0,1,1,1
Luis,M,1,1,1,0,1,1,1,1,1,0,1,1,1


In [49]:
# Estima género a partir del nombre usando NameDataset
df['genero_es'] = df['nombre_pila'].apply(lambda x: NameWrapper(nd.search(x)).gender)
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0,Ana,Female
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0,Luis,Male
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,None,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0,María,Female
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,Male
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,None,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,Male
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0,Elena,Female
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0,Gorka,Male


In [50]:
# Comprueba consistencia entre genero (F/M) y genero_es (Female/Male)
df_correcto = (
    ((df['genero'] == 'F') & (df['genero_es'] == 'Female')) |
    ((df['genero'] == 'M') & (df['genero_es'] == 'Male'))
)
df_correcto.head()

0     True
1     True
3    False
4    False
5    False
dtype: bool

In [51]:
# Creo un df para testear las filas inconsistentes o vacias (negación de las correctas)
df_test = df[~df_correcto]
df_test

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,None,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0,María,Female
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,Male
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,None,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,Male


In [52]:
# El primer paso, sería completar 'genero' faltante a partir del sexo biologico
mapeo_sexo = {'Female': 'F', 'Male': 'M'}
df['genero'] = df['genero'].fillna(df['sexo_biologico'])
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0,Ana,Female
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0,Luis,Male
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,F,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0,María,Female
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,Male
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,None,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,Male
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0,Elena,Female
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0,Gorka,Male


In [53]:
# En segundo, sería completar 'genero' faltante a partir del genero estimado
mapeo_sexo = {'Female': 'F', 'Male': 'M'}
df['genero_es'] = df['genero_es'].map(mapeo_sexo)
df['genero'] = df['genero'].fillna(df['genero_es'])
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0,Ana,F
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0,Luis,M
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,F,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0,María,F
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,M
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,M,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,M
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0,Elena,F
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0,Gorka,M


In [54]:
# Volvemos a chequear diferencias tras completar 'genero'
df_match = (
    ((df['genero'] == 'F') & (df['genero_es'] == 'F')) |
    ((df['genero'] == 'M') & (df['genero_es'] == 'M'))
)
df_test = df[~df_match]
df_test.head()

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,M


In [55]:
# Volvemos a chequear diferencias tras completar 'genero'
df_match = (
    ((df['sexo_biologico'] == 'F') & (df['genero_es'] == 'F')) |
    ((df['sexo_biologico'] == 'M') & (df['genero_es'] == 'M'))
)
df_test = df[~df_match]
df_test.head()

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
1,1985-12-03,2019-12-03,Luis Rodríguez,None,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0,Luis,M
5,1981-03-11,2018-04-10,Luis González,None,48007,1248870G,M,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,M


In [56]:
# Por ultimo, rellenamos el sexo biológico con el genero una vez tenemos todos los campos.
df['sexo_biologico'] = df['sexo_biologico'].fillna(df['genero'])
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0,Ana,F
1,1985-12-03,2019-12-03,Luis Rodríguez,M,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0,Luis,M
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,F,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0,María,F
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,M
5,1981-03-11,2018-04-10,Luis González,M,48007,1248870G,M,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,M
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0,Elena,F
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0,Gorka,M


In [57]:
# Información del DataFrame tras ajustes de género
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 8
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fecha_nacimiento  7 non-null      datetime64[ns]
 1   fecha_alta        7 non-null      datetime64[ns]
 2   nombre            7 non-null      object        
 3   sexo_biologico    7 non-null      object        
 4   codigo_postal     7 non-null      int64         
 5   nif               7 non-null      object        
 6   genero            7 non-null      object        
 7   departamento      7 non-null      object        
 8   activo            7 non-null      bool          
 9   horas_semanales   7 non-null      object        
 10  bonus             4 non-null      float64       
 11  pago              7 non-null      float64       
 12  bonus1            7 non-null      float64       
 13  bonus2            7 non-null      float64       
 14  nombre_pila       7 non-null      o

In [58]:
# Recuento de nulos tras ajustes de género
df.isnull().sum()

fecha_nacimiento    0
fecha_alta          0
nombre              0
sexo_biologico      0
codigo_postal       0
nif                 0
genero              0
departamento        0
activo              0
horas_semanales     0
bonus               3
pago                0
bonus1              0
bonus2              0
nombre_pila         0
genero_es           0
dtype: int64

In [59]:
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0,Ana,F
1,1985-12-03,2019-12-03,Luis Rodríguez,M,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0,Luis,M
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,F,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0,María,F
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,M
5,1981-03-11,2018-04-10,Luis González,M,48007,1248870G,M,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,M
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0,Elena,F
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0,Gorka,M


In [60]:
# Reseteo el index eliminado el index anterior
df.reset_index(drop=True)

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0,Ana,F
1,1985-12-03,2019-12-03,Luis Rodríguez,M,41001,17905733A,M,IT,True,35h,NaN,611.0,5250.0,6500.0,Luis,M
2,1992-07-22,2021-07-22,María López,F,48004,52745245N,F,Finanzas,False,40h,3000.0,1074.0,3000.0,3000.0,María,F
3,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,M
4,1981-03-11,2018-04-10,Luis González,M,48007,1248870G,M,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,M
5,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0,Elena,F
6,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0,Gorka,M


In [61]:
# Convierte 'horas_semanales' de texto (e.g. '40h') a entero
df['horas_semanales'] = df['horas_semanales'].str.rstrip('h').astype(int)
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago,bonus1,bonus2,nombre_pila,genero_es
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0,5000.0,5000.0,Ana,F
1,1985-12-03,2019-12-03,Luis Rodríguez,M,41001,17905733A,M,IT,True,35,NaN,611.0,5250.0,6500.0,Luis,M
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,F,Finanzas,False,40,3000.0,1074.0,3000.0,3000.0,María,F
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0,5250.0,5000.0,Carlos,M
5,1981-03-11,2018-04-10,Luis González,M,48007,1248870G,M,IT,True,45,6000.0,0.0,6000.0,6000.0,Luis,M
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0,7000.0,7000.0,Elena,F
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,0.0,5250.0,3000.0,Gorka,M


In [62]:
# Verifica nulos pendientes
df.isnull().sum()

fecha_nacimiento    0
fecha_alta          0
nombre              0
sexo_biologico      0
codigo_postal       0
nif                 0
genero              0
departamento        0
activo              0
horas_semanales     0
bonus               3
pago                0
bonus1              0
bonus2              0
nombre_pila         0
genero_es           0
dtype: int64

In [63]:
# Selección y renombrado de columnas para el dataset final
def selector_columnas(df: pd.DataFrame) -> pd.DataFrame:
    # Columnas a conservar
    cols_keep = [
        'nif',
        'fecha_nacimiento',
        'fecha_alta',
        'sexo_biologico',
        'activo',
        'codigo_postal',
        'bonus2',
    ]
    df = df[cols_keep]
    # Renombra a nombres de salida
    df = df.rename(columns={
        'nif':'IdAsegurado',
        'fecha_nacimiento':'FNacim',
        'fecha_alta':'FAlta',
        'sexo_biologico':'Sexo',
        'activo':'Activo',
        'codigo_postal':'CP',
        'bonus2':'Capital',
    })
    return df

In [64]:
# Aplica la función de selección y renombrado
df = selector_columnas(df)
df

,IdAsegurado,FNacim,FAlta,Sexo,Activo,CP,Capital
0,88862218Q,1990-01-15,2020-01-15,F,True,28013,5000.0
1,17905733A,1985-12-03,2019-12-03,M,True,41001,6500.0
3,52745245N,1992-07-22,2021-07-22,F,False,48004,3000.0
4,82167366G,1988-04-10,2018-04-10,M,True,48001,5000.0
5,1248870G,1981-03-11,2018-04-10,M,True,48007,6000.0
7,28859641T,1995-11-08,2022-11-08,F,True,15003,7000.0
8,11859641K,1986-05-08,1922-03-04,M,True,46001,3000.0


In [65]:
# Exporta el dataset limpio a Feather
df.to_feather('../data/02_intermediate/empleados_metacortex_clean.ftr')